# Classification: CATS vs DOGS 
    jpeg -> SqueezeNET Keras -> [0,1]
---

Import Tensorflow with Keras and Packages

In [1]:
import tensorflow as tf

In [2]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Input, GlobalAveragePooling2D
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
from keras import backend as K
from keras.optimizers import SGD

In [4]:
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from PIL import Image

In [5]:
from matplotlib import pyplot as plt
%matplotlib inline 

In [6]:
import os

---
### FOLDERS and CATEGORIES:

- Folders

In [7]:
datadir = 'dogVScat'
cat1 = 'dogs'
cat2 = 'cats'

- Training set

In [8]:
target = os.chdir(datadir + "/train/" + cat1)
cat_1 = os.listdir()

In [9]:
target = os.chdir("../" + cat2)
cat_2 = os.listdir()

In [10]:
print("CAT1:",len(cat_1),"CAT2:",len(cat_2), "TOTAL:",len(cat_1)+len(cat_2))

CAT1: 5501 CAT2: 6054 TOTAL: 11555


- Validation set

In [11]:
target = os.chdir("../../validation/" + cat1)
cat_1_val = os.listdir()

In [12]:
target = os.chdir("../" + cat2)
cat_2_val = os.listdir()

In [13]:
print("CAT1:",len(cat_1_val),"CAT2:",len(cat_2_val), "TOTAL:",len(cat_1_val)+len(cat_2_val))

CAT1: 1302 CAT2: 1500 TOTAL: 2802


- Return to Home

In [14]:
os.chdir("../../../")

---
### HYPER PARAMETERS:

- Dimensions
- Epochs
- Batch
- Directories
- Learning Rate
- Decadimento 
- Momento

In [15]:
### CNN 

img_width, img_height = 300, 300

data_dir = 'dogVScat'
train_data_dir = data_dir + '/train'
validation_data_dir = data_dir + '/validation'
test_data_dir = data_dir + '/test'

nb_train_samples = len(cat_1)+len(cat_2)
nb_validation_samples = len(cat_1_val)+len(cat_2_val)
epochs = 2                                                # 50
batch_size = 32                                           # 16

---
### CNN MODEL:

- Input Layer

In [17]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

- Sequential Layers Architecture

In [18]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 298, 298, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 298, 298, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 147, 147, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 147, 147, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxP

In [19]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

- <a href="https://ai.google/research/pubs/pub43022">Inception</a> Layers Architecture

In [20]:
input_image = Input(shape = input_shape)

In [21]:
depth_cnn = 32  
# depth_cnn = 64

In [22]:
# REWRITE with separated activation function 

tower_1 = Conv2D(depth_cnn, (1,1), padding='same')(input_image)
tower_1 = Activation('relu')(tower_1)
tower_1 = Conv2D(depth_cnn, (3,3), padding='same')(tower_1)
tower_1 = Activation('relu')(tower_1)

tower_2 = Conv2D(depth_cnn, (1,1), padding='same')(input_image)
tower_2 = Activation('relu')(tower_2)
tower_2 = Conv2D(depth_cnn, (5,5), padding='same')(tower_2)
tower_2 = Activation('relu')(tower_2)

tower_3 = MaxPooling2D((3,3), strides=(1,1), padding='same')(input_image)
tower_3 = Conv2D(depth_cnn, (1,1), padding='same')(tower_3)
tower_3 = Activation('relu')(tower_3)

output = keras.layers.concatenate([tower_1, tower_2, tower_3], axis = 3)

output = Flatten()(output)
out = Dropout(0.5)(output)
out = Dense(1)(out)
out = Activation('sigmoid')(out)

In [23]:
model_inc = Model(inputs=input_image, outputs=out)
print(model_inc.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 300, 300, 32) 128         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 300, 300, 32) 128         input_1[0][0]                    
__________________________________________________________________________________________________
activation_6 (Activation)       (None, 300, 300, 32) 0           conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation

In [24]:
model_inc.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

- <a href="https://en.wikipedia.org/wiki/SqueezeNet">SqueezeNet</a> Layers Architecture

In [25]:
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

In [26]:
# Modular function for Fire Node

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    if K.image_data_format() == 'channels_first':
        channel_axis = 1
    else:
        channel_axis = 3
    
    x = Conv2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Conv2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Conv2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = keras.layers.concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

In [27]:
x = Conv2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(input_image)
x = Activation('relu', name='relu_conv1')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

x = fire_module(x, fire_id=2, squeeze=16, expand=64)
x = fire_module(x, fire_id=3, squeeze=16, expand=64)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

x = fire_module(x, fire_id=4, squeeze=32, expand=128)
x = fire_module(x, fire_id=5, squeeze=32, expand=128)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

x = fire_module(x, fire_id=6, squeeze=48, expand=192)
x = fire_module(x, fire_id=7, squeeze=48, expand=192)
x = fire_module(x, fire_id=8, squeeze=64, expand=256)
x = fire_module(x, fire_id=9, squeeze=64, expand=256)

x = Dropout(0.5, name='drop9')(x)

x = Conv2D(1, (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('sigmoid', name='loss')(x)

In [28]:
model_sqz = Model(inputs=input_image, outputs=x)
print(model_sqz.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 149, 149, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 149, 149, 64) 0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 74, 74, 64)   0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [29]:
model_sqz.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

- Data Generators

In [30]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [31]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

- Train VS Validation SET

In [32]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 11555 images belonging to 2 classes.


In [33]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2802 images belonging to 2 classes.


- Model Training 

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
history_inc = model_inc.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [34]:
history_sqz = model_sqz.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
361/361 [==============================] - 571s 2s/step - loss: 0.6931 - acc: 0.5236 - val_loss: 0.6931 - val_acc: 0.5341
Epoch 2/2
361/361 [==============================] - 576s 2s/step - loss: 0.6931 - acc: 0.5233 - val_loss: 0.6931 - val_acc: 0.5368


- Save

In [36]:
model_sqz.save_weights('keras_sqz_weights.h5')
model_sqz.save('keras_sqz_model.h5') 